In [81]:
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import time
import math
import dask.array as da
import graphviz

In [33]:
def read_MODIS_level2_data(MOD06_file,MOD03_file):
    print(MOD06_file)
    print(MOD03_file)
    print('reading the cloud mask from MOD06_L2 product')
    myd06 = Dataset(MOD06_file, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
    CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    CM = np.ravel(CM) # Changing The Shape Of The Variable 'Longitude'.
    print('level-2 cloud mask array shape',CM.shape)

    myd03 = Dataset(MOD03_file, "r")
    print('reading the lat-lon from MOD03 product')
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    latitude = np.ravel(latitude) # Changing The Shape Of The Variable 'Latitude'.
    latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    longitude = np.ravel(longitude) # Changing The Shape Of The Variable 'Longitude'.
    longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    print('level-2 lat-lon array shape',latitude.shape)

    return latitude,longitude,CM

In [34]:
def value_locate(refx, x):
    refx = np.array(refx)
    x = np.array(x)
    loc = np.zeros(len(x), dtype='int')

    for i in range(len(x)):
        ix = x[i]
        ind = ((refx - ix) <= 0).nonzero()[0]
        if len(ind) == 0:
            loc[i] = -1
        else: loc[i] = ind[-1]

    return loc

def division(n, d):

    div = np.zeros(len(d))
    for i in range(len(d)):
        if d[i] >0:
          div[i]=n[i]/d[i]
        else: div[i]=None 

    return div

In [35]:
MOD03_path = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
MOD06_path = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
satellite = 'Aqua'

yr = [2008]
mn = [1] #np.arange(1,13)  #[1]
dy = [1] #np.arange(1,32) # [1] #np.arange(1,31)
# latitude and longtitude boundaries of level-3 grid
lat_bnd = np.arange(-90,91,1)
lon_bnd = np.arange(-180,180,1)
nlat = 180
nlon = 360

TOT_pix      = np.zeros(nlat*nlon)
CLD_pix      = np.zeros(nlat*nlon)

In [36]:
MOD03_fp = 'MYD03.A*.hdf'
MOD06_fp = 'MYD06_L2.A*.hdf'
MOD03_fn, MOD06_fn =[],[]
MOD03_fn2, MOD06_fn2 =[],[]
for MOD06_flist in  os.listdir(MOD06_path):
    if fnmatch.fnmatch(MOD06_flist, MOD06_fp):
        MOD06_fn = MOD06_flist
        MOD06_fn2.append(MOD06_flist)
for MOD03_flist in  os.listdir(MOD03_path):
    if fnmatch.fnmatch(MOD03_flist, MOD03_fp):
        MOD03_fn = MOD03_flist
        MOD03_fn2.append(MOD03_flist)
if MOD03_fn and MOD06_fn:
    # if both MOD06 and MOD03 products are in the directory
    print('reading level 2 geolocation and cloud data')
    print(MOD06_fn)
    print(MOD03_fn)
    Lat,Lon,CM = read_MODIS_level2_data(MOD06_path+MOD06_fn,MOD03_path+MOD03_fn)

reading level 2 geolocation and cloud data
MYD06_L2.A2008001.0005.006.2013341193207.hdf
MYD03.A2008001.0000.006.2012066122450.hdf
/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/MYD06_L2.A2008001.0005.006.2013341193207.hdf
/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/MYD03.A2008001.0000.006.2012066122450.hdf
reading the cloud mask from MOD06_L2 product
level-2 cloud mask array shape (2748620,)
reading the lat-lon from MOD03 product
level-2 lat-lon array shape (2748620,)


In [37]:
%time
Lat

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11.2 µs


array([69.99062 , 69.99425 , 69.99774 , ..., 47.48759 , 47.469513,
       47.45127 ], dtype=float32)

In [38]:
Lat.shape

(2748620,)

In [39]:
Lon.shape

(2748620,)

In [40]:
CM.shape

(2748620,)

In [41]:
#for items in MOD06_fn2:

In [42]:
myd06_name = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
cm = np.array(2)
cm1 = np.array(2)
cmr = np.array(2)
for MOD06_file in MOD06_fn2:
    MOD06_file2 = myd06_name + MOD06_file
    myd06 = Dataset(MOD06_file2, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:]# Reading Specific Variable 'Cloud_Mask_1km'.
    CMr = myd06.variables["Cloud_Mask_1km"][:,:,0]
    CM1   = (np.array(CM[:,:,0],dtype='byte') & 0b00000001)
    CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    #CM = np.array(CM).byteswap().newbyteorder()
    CM = np.ravel(CM) # Changing The Shape Of The Variable 'Longitude'.
    #ar.append(CM)
    cm = np.append(cm, CM)
    cm1 = np.append(cm1, CM1)
    cmr = np.append(cmr, CMr)
    #ar.insert(CM)
    print('level-2 cloud mask array shape',CM.shape)
    

level-2 cloud mask array shape (2748620,)
level-2 cloud mask array shape (2748620,)
level-2 cloud mask array shape (2748620,)


In [43]:
myd03_name = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
lat = np.array(2)
lon = np.array(2)
for MOD03_file in MOD03_fn2:
    MOD03_file2 = myd03_name + MOD03_file
    myd03 = Dataset(MOD03_file2, "r")
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    latitude = np.ravel(latitude) # Changing The Shape Of The Variable 'Latitude'.
    #latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    lat = np.append(lat, latitude)
    print('Latitude Shape Is: ',lat.shape)

    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    longitude = np.ravel(longitude) # Changing The Shape Of The Variable 'Longitude'.
    #longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    lon = np.append(lon, longitude)
    print('Latitude Shape Is: ',lon.shape)


    

Latitude Shape Is:  (2748621,)
Latitude Shape Is:  (2748621,)
Latitude Shape Is:  (5497241,)
Latitude Shape Is:  (5497241,)
Latitude Shape Is:  (8245861,)
Latitude Shape Is:  (8245861,)


In [44]:
MOD06_file2

'/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/MYD06_L2.A2008001.0005.006.2013341193207.hdf'

In [45]:
cm.shape

(8245861,)

In [46]:
cm

array([2, 0, 0, ..., 0, 0, 0])

In [47]:
lat.shape

(8245861,)

In [48]:
MOD03_fn2

['MYD03.A2008001.0010.006.2012066122416.hdf',
 'MYD03.A2008001.0005.006.2012066122516.hdf',
 'MYD03.A2008001.0000.006.2012066122450.hdf']

In [49]:
lon.size

8245861

In [50]:
CMB123 = lat.astype(np.int8)
CMB123 = xr.DataArray(CMB123)
CMB124 = lon.astype(np.int8)
CMB124 = xr.DataArray(CMB124)

In [51]:
#

dsa3 = xr.Dataset()
dsa3.coords['Latitude'] = (('x'), lat)
dsa3.coords['Longitude'] = (('x'), lon)
dsa3.coords['Bit0'] = (('x'), cm1)
dsa3.coords['Bit12'] = (('x'), cm)
dsa3.coords['LatInt'] = (('x'), CMB123)
dsa3.coords['LonInt'] = (('x'), CMB124)

In [52]:
dsa3

<xarray.Dataset>
Dimensions:    (x: 8245861)
Coordinates:
    Latitude   (x) float64 2.0 34.24 34.24 34.24 ... 47.51 47.49 47.47 47.45
    Longitude  (x) float64 2.0 14.01 14.06 14.11 ... 47.61 47.67 47.72 47.78
    Bit0       (x) int64 2 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1 1
    Bit12      (x) int64 2 0 0 0 0 0 0 0 0 3 0 2 3 ... 0 0 0 0 0 0 0 0 0 0 0 0 0
    LatInt     (x) int8 2 34 34 34 34 34 34 34 34 ... 47 47 47 47 47 47 47 47 47
    LonInt     (x) int8 2 14 14 14 14 14 14 14 14 ... 47 47 47 47 47 47 47 47 47
Dimensions without coordinates: x
Data variables:
    *empty*

In [113]:
CMrd = cmr.data
CMrd = np.ravel(CMrd)

In [114]:
dsa3['CloudFraction'] = (('x'), CMrd)

In [115]:
dsa3

<xarray.Dataset>
Dimensions:        (x: 8245861)
Coordinates:
    Latitude       (x) float64 2.0 34.24 34.24 34.24 ... 47.51 47.49 47.47 47.45
    Longitude      (x) float64 2.0 14.01 14.06 14.11 ... 47.61 47.67 47.72 47.78
    Bit0           (x) int64 2 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    Bit12          (x) int64 dask.array<shape=(8245861,), chunksize=(8245861,)>
    LatInt         (x) int8 2 34 34 34 34 34 34 34 ... 47 47 47 47 47 47 47 47
    LonInt         (x) int8 2 14 14 14 14 14 14 14 ... 47 47 47 47 47 47 47 47
Dimensions without coordinates: x
Data variables:
    CloudFraction  (x) int64 2 49 49 49 49 49 49 ... -79 -79 -79 -79 -79 -79 -79

In [56]:
df = dsa3.to_dask_dataframe()

In [67]:
df3 = df.persist()

In [74]:
df5 = da.from_array(dsa3, chunks =(8245861,))

AttributeError: 'Dataset' object has no attribute 'shape'

In [75]:
df6 = da.from_array(cm, chunks =(8245861,))

In [76]:
df6

dask.array<array, shape=(8245861,), dtype=int64, chunksize=(8245861,)>

In [92]:
df6.to_dask_dataframe()

Dask Series Structure:
npartitions=1
0          int64
8245860      ...
dtype: int64
Dask Name: from-dask, 2 tasks

In [97]:
df6.compute()

array([2, 0, 0, ..., 0, 0, 0])

In [101]:
df6c = df6.compute()

In [98]:
df6

dask.array<array, shape=(8245861,), dtype=int64, chunksize=(8245861,)>

In [104]:
#

dsa3 = xr.Dataset()
dsa3.coords['Latitude'] = (('x'), lat)
dsa3.coords['Longitude'] = (('x'), lon)
dsa3.coords['Bit0'] = (('x'), cm1)
dsa3.coords['Bit12'] = (('x'), df6)
dsa3.coords['LatInt'] = (('x'), CMB123)
dsa3.coords['LonInt'] = (('x'), CMB124)

In [105]:
dsa3

<xarray.Dataset>
Dimensions:    (x: 8245861)
Coordinates:
    Latitude   (x) float64 2.0 34.24 34.24 34.24 ... 47.51 47.49 47.47 47.45
    Longitude  (x) float64 2.0 14.01 14.06 14.11 ... 47.61 47.67 47.72 47.78
    Bit0       (x) int64 2 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1 1
    Bit12      (x) int64 dask.array<shape=(8245861,), chunksize=(8245861,)>
    LatInt     (x) int8 2 34 34 34 34 34 34 34 34 ... 47 47 47 47 47 47 47 47 47
    LonInt     (x) int8 2 14 14 14 14 14 14 14 14 ... 47 47 47 47 47 47 47 47 47
Dimensions without coordinates: x
Data variables:
    *empty*

In [107]:
dsa3.Bit12.values

array([2, 0, 0, ..., 0, 0, 0])

In [65]:
df2 = dsa3.Bit0.to_dask_dataframe()

AttributeError: 'DataArray' object has no attribute 'to_dask_dataframe'

In [ ]:
df5 = dsa3.to_dask_dataframe()

In [116]:
%time
dsa4 = list(dsa3.groupby('LatInt', 'LonInt'))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


In [26]:
dsa4.index

<function list.index>

In [109]:
%time
dsa4

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


[(2, <xarray.Dataset>
  Dimensions:    (x: 1)
  Coordinates:
      Latitude   (x) float64 2.0
      Longitude  (x) float64 2.0
      Bit0       (x) int64 2
      Bit12      (x) int64 dask.array<shape=(1,), chunksize=(1,)>
      LatInt     (x) int8 2
      LonInt     (x) int8 2
  Dimensions without coordinates: x
  Data variables:
      *empty*), (13, <xarray.Dataset>
  Dimensions:    (x: 6702)
  Coordinates:
      Latitude   (x) float64 14.0 13.99 13.98 14.0 ... 13.15 13.15 13.14 13.13
      Longitude  (x) float64 32.93 32.87 32.92 32.74 ... 32.56 32.61 32.65 32.69
      Bit0       (x) int64 1 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1 1
      Bit12      (x) int64 dask.array<shape=(6702,), chunksize=(6702,)>
      LatInt     (x) int8 13 13 13 13 13 13 13 13 13 ... 13 13 13 13 13 13 13 13
      LonInt     (x) int8 32 32 32 32 32 32 32 32 32 ... 32 32 32 32 32 32 32 32
  Dimensions without coordinates: x
  Data variables:
      *empty*), (14, <xarray.Dataset>
  Dimensions:    (x

In [110]:
list1=[]
for key, value in dsa4:
    #print(key)
    list1.append(key)

In [111]:
Nobs = np.sum(cm1>=0)
print("Number Of Total Pixel: ", np.sum(cm1>=0), " Pixels")
print("Number Of Pixel With Cloud Mask Determined: ", np.sum(cm1==1), " Pixels")

Number Of Total Pixel:  8245861  Pixels
Number Of Pixel With Cloud Mask Determined:  8245860  Pixels


In [112]:
%time
i = 0
while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[0][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = dsa81.groupby('Bit12').count()/Nobs*100
    print(dsa91)
    i += 1

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


/Users/saviosebastian/anaconda3/lib/python3.6/site-packages/xarray/core/groupby.py:723: FutureWarning: Default reduction dimension will be changed to the grouped dimension after xarray 0.12. To silence this warning, pass dim=xarray.ALL_DIMS explicitly.
  **kwargs)


<xarray.Dataset>
Dimensions:  (Bit12: 1)
Coordinates:
  * Bit12    (Bit12) int64 2
Data variables:
    *empty*
<xarray.Dataset>
Dimensions:  (Bit12: 1)
Coordinates:
  * Bit12    (Bit12) int64 2
Data variables:
    *empty*
<xarray.Dataset>
Dimensions:  (Bit12: 1)
Coordinates:
  * Bit12    (Bit12) int64 2
Data variables:
    *empty*
<xarray.Dataset>
Dimensions:  (Bit12: 1)
Coordinates:
  * Bit12    (Bit12) int64 2
Data variables:
    *empty*
<xarray.Dataset>
Dimensions:  (Bit12: 1)
Coordinates:
  * Bit12    (Bit12) int64 2
Data variables:
    *empty*
<xarray.Dataset>
Dimensions:  (Bit12: 1)
Coordinates:
  * Bit12    (Bit12) int64 2
Data variables:
    *empty*
<xarray.Dataset>
Dimensions:  (Bit12: 1)
Coordinates:
  * Bit12    (Bit12) int64 2
Data variables:
    *empty*


KeyboardInterrupt: 

In [31]:
%time
i = 0
start_time = time.time()
while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = dsa81.groupby('Bit12').count()/Nobs*100
    print(dsa91)
    i += 1
end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
%time

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs
<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 2
Data variables:
    CloudFraction  (Bit12) float64 1.213e-05
<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 3
Data variables:
    CloudFraction  (Bit12) float64 0.004511
<xarray.Dataset>
Dimensions:        (Bit12: 2)
Coordinates:
  * Bit12          (Bit12) int64 2 3
Data variables:
    CloudFraction  (Bit12) float64 1.213e-05 0.0002911
<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 3
Data variables:
    CloudFraction  (Bit12) float64 0.0001455
<xarray.Dataset>
Dimensions:        (Bit12: 4)
Coordinates:
  * Bit12          (Bit12) int64 0 1 2 3
Data variables:
    CloudFraction  (Bit12) float64 2.425e-05 0.0005579 0.0005942 0.01359
<xarray.Dataset>
Dimensions:        (Bit12: 3)
Coordinates:
  * Bit12          (Bit12) int64 1 2 3
Data variabl

In [187]:
%time
i = 0
while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = dsa81.groupby('Bit12').count()
    print(dsa91)
    i += 1

<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 2
Data variables:
    CloudFraction  (Bit12) int64 1
<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 3
Data variables:
    CloudFraction  (Bit12) int64 372
<xarray.Dataset>
Dimensions:        (Bit12: 2)
Coordinates:
  * Bit12          (Bit12) int64 2 3
Data variables:
    CloudFraction  (Bit12) int64 1 24
<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 3
Data variables:
    CloudFraction  (Bit12) int64 12
<xarray.Dataset>
Dimensions:        (Bit12: 4)
Coordinates:
  * Bit12          (Bit12) int64 0 1 2 3
Data variables:
    CloudFraction  (Bit12) int64 2 46 49 1121
<xarray.Dataset>
Dimensions:        (Bit12: 3)
Coordinates:
  * Bit12          (Bit12) int64 1 2 3
Data variables:
    CloudFraction  (Bit12) int64 2 36 1150
<xarray.Dataset>
Dimensions:        (Bit12: 4)
Coordinates:
  * Bit12          (Bit12) 

In [198]:
time.time()

1546942482.3784

In [117]:
%time
start_time = time.time()
i = 0

while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    print(dsa91)
    i += 1

end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
[(2, <xarray.Dataset>
Dimensions:        (x: 1)
Coordinates:
    Latitude       (x) float64 2.0
    Longitude      (x) float64 2.0
    Bit0           (x) int64 2
    Bit12          (x) int64 dask.array<shape=(1,), chunksize=(1,)>
    LatInt         (x) int8 2
    LonInt         (x) int8 2
Dimensions without coordinates: x
Data variables:
    CloudFraction  (x) int64 2)]
[(3, <xarray.Dataset>
Dimensions:        (x: 372)
Coordinates:
    Latitude       (x) float64 14.0 13.99 13.99 14.0 ... 13.84 13.84 13.83 13.83
    Longitude      (x) float64 28.95 28.97 28.99 28.88 ... 28.94 28.96 28.98
    Bit0           (x) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    Bit12          (x) int64 dask.array<shape=(372,), chunksize=(372,)>
    LatInt         (x) int8 13 13 13 13 13 13 13 13 ... 13 13 13 13 13 13 13 13
    LonInt         (x) int8 28 28 28 28 28 28 28 28 ... 28 28 28 28 28 28 28 28
Dimensions without coordinate

In [118]:
from dask.distributed import Client
client = Client()

In [119]:
client.cluster

In [122]:
from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client()
client

Client Scheduler: tcp://127.0.0.1:54707,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [123]:
%time
start_time = time.time()
i = 0

while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    print(dsa91)
    i += 1

end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 11 µs
[(2, <xarray.Dataset>
Dimensions:        (x: 1)
Coordinates:
    Latitude       (x) float64 2.0
    Longitude      (x) float64 2.0
    Bit0           (x) int64 2
    Bit12          (x) int64 dask.array<shape=(1,), chunksize=(1,)>
    LatInt         (x) int8 2
    LonInt         (x) int8 2
Dimensions without coordinates: x
Data variables:
    CloudFraction  (x) int64 2)]
[(3, <xarray.Dataset>
Dimensions:        (x: 372)
Coordinates:
    Latitude       (x) float64 14.0 13.99 13.99 14.0 ... 13.84 13.84 13.83 13.83
    Longitude      (x) float64 28.95 28.97 28.99 28.88 ... 28.94 28.96 28.98
    Bit0           (x) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    Bit12          (x) int64 dask.array<shape=(372,), chunksize=(372,)>
    LatInt         (x) int8 13 13 13 13 13 13 13 13 ... 13 13 13 13 13 13 13 13
    LonInt         (x) int8 28 28 28 28 28 28 28 28 ... 28 28 28 28 28 28 28 28
Dimensions without coordina